## Check that CustomDataModule works

The reason I want to check this is that the augmented images that i logged through tensorboard were completely off. So i wanted to make sure, that the CustomDataModule class works properly. Either it is the dataloaders not working or the images are used wrong in my model.py

1. Instantiate the CustomDataModule and its .setup and .train_dataloader to be able to extract tensor batches from it
2. Create a data variable containing images as tensors (of shape torch.Size([16, 3, 224, 224]))
3. Convert back to actual image using ToPILImage
4. Display it with PIL

So i manually sample from my Dataset and convert the image back to an actual image, then display it with PIL
If it displays an image you recognise then you're not loading images wrong, you might just be using them wrong in the model


The code below only works if the dataloaders in customDataModule does not normalize the tensor.


In [1]:
import config
from customDataModule import CustomDataModule
import torchvision.transforms as T
import torch
from PIL import Image
import numpy as np

# Instantiate class

dm = CustomDataModule(
        data_dir=config.DATA_DIR,
        train_csv=config.TRAIN_CSV,
        val_csv=config.VAL_CSV,
        test_csv=config.VAL_CSV,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )

## Instantiate objects
dm.setup("train") # setup skal instantiates for at kunne lave train_dataloader
dm.train_dataloader()

# Take out the first batch of tensor images
train_dataloader = dm.train_dataloader()
train_dataloader_iterator = iter(train_dataloader) # Use iter() to generate an iterator for the dataloader
data = next(train_dataloader_iterator) # use next() function on the iterator object to get 1st batch of data
data
print(data[0].shape)
#data # "data will now contain the first batch of data from train_dataloader, 
     # which may include one or multiple images depending on the batch size. If the batch size is 16, data will have the shape [16, C, H, W], where C, H, and W are the dimensions of the image in channel, height, and width, respectively."


# Convert tensor to PIL image

## Get first image from batch
image_tensor = data[0][1]   # data should have shape 16: [16, C, H, W]
print(image_tensor.shape)

## define a transform to convert a tensor to PIL image
transform = T.ToPILImage()

## convert the tensor to PIL image using above transform
image_tensor = transform(image_tensor)

## display the PIL image

image_tensor.show()

torch.Size([16, 3, 224, 224])
torch.Size([3, 224, 224])


# Create 3 different .csv files for train, valid and test (while also randomly splitting the dataset)

In [2]:
from createCSV import createCSV
import config
csv = createCSV(
        base_dir = config.BASE_DIR,
        all_csv = config.ALL_CSV,
        train_csv = config.TRAIN_CSV,
        val_csv = config.VAL_CSV,
        test_csv = config.TEST_CSV,
        label_column = config.LABEL_COLUMN,
        test_size = config.TEST_SIZE,
        seed = config.SEED)
csv.set_all_seeds()
csv.df()

['All', 'Approved', 'NonApproved', 'test_set.csv', 'train_set.csv', 'val_set.csv', 'xlbst.csv']
                                           file_name  label parcel_id
0   22-0223605_F78EFF88702EA742E0530EEE260AEFC6.jpeg      1   0223605
1   22-0223605_F78EFF887030A742E0530EEE260AEFC6.jpeg      1   0223605
2   22-0223605_F78EFF88703AA742E0530EEE260AEFC6.jpeg      1   0223605
28  22-0225160_F78EFF887012A742E0530EEE260AEFC6.jpeg      1   0225160
29  22-0225160_F78EFF887013A742E0530EEE260AEFC6.jpeg      1   0225160
30  22-0225160_F78EFF887014A742E0530EEE260AEFC6.jpeg      1   0225160
31  22-0225160_F78EFF887015A742E0530EEE260AEFC6.jpeg      1   0225160
32  22-0225160_F78EFF887016A742E0530EEE260AEFC6.jpeg      1   0225160
33  22-0225160_F78EFF887017A742E0530EEE260AEFC6.jpeg      1   0225160
34  22-0225160_F78EFF88701AA742E0530EEE260AEFC6.jpeg      1   0225160
35  22-0225160_F78EFF88701BA742E0530EEE260AEFC6.jpeg      1   0225160
36  22-0225160_F78EFF88701CA742E0530EEE260AEFC6.jpeg      1   02

# Create model using NN class (LightningModule), create CustomDataModule (LightningDataModule), set up Trainer, and then fit, validate and test

In [1]:
# Train model

from model import NN
from customDataModule import CustomDataModule
import config
from callbacks import MyPrintingCallback, EarlyStopping
import pytorch_lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch import seed_everything
from pytorchModel import *
import torch

seed_everything(
    42, workers=True
)  # By setting workers=True in seed_everything(), Lightning derives unique seeds across all dataloader workers and processes for torch, numpy and stdlib random number generators. When turned on, it ensures that e.g. data augmentations are not repeated across workers.



if __name__ == "__main__":
    
    pytorch_model = pytorchModel(num_classes=config.NUM_CLASSES)
    
    
    logger = TensorBoardLogger("tb_logs", name="my_model") # tb_logs is the folder, name is the name of the experiment/model
    model = NN(
        model=pytorch_model,
        input_size=config.IN_CHANNELS,
        num_classes=config.NUM_CLASSES,
        learning_rate=config.LEARNING_RATE,
    )  # .to(device)
    dm = CustomDataModule(
        data_dir=config.DATA_DIR,
        train_csv=config.TRAIN_CSV,
        val_csv=config.VAL_CSV,
        test_csv=config.VAL_CSV,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        logger=logger, # PyTorch lightning will automatically know what we are logging by looking at our model.py logs
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=config.MIN_EPOCHS,
        max_epochs=config.MAX_EPOCHS,
        deterministic=config.DETERMINISTIC#,
#        callbacks=[MyPrintingCallback(), EarlyStopping(monitor="val_loss")],
    )  # deterministic ensures random seed reproducibility

    trainer.fit(model, dm)  # it will automatically know which dataloader to use
    trainer.validate(model, dm)
    trainer.test(model, dm)

# A general place to start is to set num_workers equal to the number of CPU cores on that machine. You can get the number of CPU cores in python using os.cpu_count(), but note that depending on your batch size, you may overflow RAM memory.


Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type                | Params
--------------------------------------------------
0 | model     | pytorchModel        | 878 K 
1 | loss_fn   | CrossEntropyLoss    | 0     
2 | f1_score  | MulticlassF1Score   | 0     
3 | accuracy  | MulticlassAccuracy  | 0     
4 | precision | MulticlassPrecision | 0     
5 | recall    | MulticlassRecall    | 0     
--------------------------------------------------
878 K     Trainable params
0         Non-trainable params
878 K     Total params
3.514     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]torch.Size([15, 3, 224, 224])


C:\Users\kaspe\.conda\envs\LBST\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 4/4 [00:08<00:00,  2.19s/it, v_num=40]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]torch.Size([15, 3, 224, 224])

Epoch 1: 100%|██████████| 4/4 [00:08<00:00,  2.12s/it, v_num=40, train_loss=6.350, train_f1_score=0.538, train_accuracy=0.538, train_precision=0.538, train_recall=0.538]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]torch.Size([15, 3, 224, 224])

Epoch 2: 100%|██████████| 4/4 [00:08<00:00,  2.13s/it, v_num=40, train_loss=0.982, train_f1_score=0.462, train_accuracy=0.462, train_precision=0.462, train_recall=0.462]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]torch.Size([15, 3, 224, 224])

Epoch 3: 100%|██████████| 4/4 [00:08<00:00,  2.03s/it, v_num=40, train_loss=0.487, train_f1_score=0.769, train_accuracy=0.769, train_precision=0.769, train_recall=0.769]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 

`Trainer.fit` stopped: `max_epochs=10` reached.


Validation DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.60it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     4.871879577636719     │
└───────────────────────────┴───────────────────────────┘

Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.74it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     4.871879577636719     │
└───────────────────────────┴───────────────────────────┘